# SPCS Networking Connectivity Test: Box.com

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Please provide your Box account subdomain (if using a custom subdomain)
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

This file contains the basic Enterprise endpoints for Box, for additional Box endpoints and firewall configuration details, see:  
https://support.box.com/hc/en-us/articles/360043696434-Configuring-A-Firewall-For-Box-Applications-and-Services

In [ ]:
# Box.com Connectivity Test Configuration

# User Configuration - UPDATE THIS if you have a custom subdomain
BOX_CUSTOM_SUBDOMAIN = ""  # Leave empty for standard Box accounts, or enter your custom subdomain (e.g., "mycompany")

# Core Box URLs
BOX_API_URL = "https://api.box.com"
BOX_APP_URL = "https://app.box.com"
BOX_UPLOAD_URL = "https://upload.box.com"
BOX_DL_URL = "https://dl.boxcloud.com"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

print(f"Configuration:")
print(f"Box API: {BOX_API_URL}")
print(f"Box App: {BOX_APP_URL}")
print(f"Box Upload: {BOX_UPLOAD_URL}")
print(f"Box Download: {BOX_DL_URL}")
if BOX_CUSTOM_SUBDOMAIN:
    print(f"Custom Subdomain: {BOX_CUSTOM_SUBDOMAIN}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\nTesting {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("BOX.COM CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['box_api'] = test_connectivity(BOX_API_URL, "Box API")
results['box_app'] = test_connectivity(BOX_APP_URL, "Box App")
results['box_upload'] = test_connectivity(BOX_UPLOAD_URL, "Box Upload")
results['box_download'] = test_connectivity(BOX_DL_URL, "Box Download")

if BOX_CUSTOM_SUBDOMAIN:
    custom_url = f"https://{BOX_CUSTOM_SUBDOMAIN}.app.box.com"
    results['box_custom'] = test_connectivity(custom_url, "Box Custom Subdomain")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "PASS" if passed else "FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\nAll connectivity tests PASSED!")
    print("Your SPCS environment can reach all required Box.com endpoints.")
    print("You can proceed with Box.com integration.")
else:
    print(f"\nSome connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for Box.com access.

The network rules below include the key Box.com endpoints required for core functionality. For additional endpoints (Box Sign, Box Education, Box for Microsoft Teams, etc.), refer to the Box firewall documentation and add them as needed.

In [ ]:
-- Create Network Rules for Box.com connectivity
-- Run this cell if connectivity tests failed

-- Network rule for Box.com core services
-- These are the key endpoints required for Box.com functionality
-- For additional endpoints (Box Sign, Box Education, Box for Microsoft Teams, etc.),
-- see https://support.box.com/hc/en-us/articles/360043696434-Configuring-A-Firewall-For-Box-Applications-and-Services

CREATE OR REPLACE NETWORK RULE box_connectivity_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = (
    -- Core Box domains
    '*.box.com:443',
    '*.app.box.com:443',
    '*.ent.box.com:443',
    '*.box.net:443',
    '*.boxcdn.net:443',
    '*.boxcloud.com:443',
    '*.services.box.com:443',
    'api.box.com:443',
    'app.box.com:443',
    'upload.box.com:443',
    'dl.boxcloud.com:443',
    'account.box.com:443'
  );

SHOW NETWORK RULES LIKE 'box_%';

In [ ]:
USE ROLE ACCOUNTADMIN;

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION box_eai
  ALLOWED_NETWORK_RULES = (
    box_connectivity_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Box.com connectivity';

GRANT USAGE ON INTEGRATION box_eai TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'box_eai';

In [ ]:
ALTER NOTEBOOK EAI_BOX
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('BOX_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.

*NOTE: Remember to restart your compute session*

In [ ]:
-- Check if network rules exist
SHOW NETWORK RULES LIKE 'box_%';

In [ ]:
-- Check if EAI exists
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'box_eai';